In [3]:
import wget
import os
import pandas as pd
import xlsxwriter
import numpy as np
import difflib
from googletrans import Translator

In [13]:
!python -m pip uninstall googletrans --yes

Found existing installation: googletrans 3.0.0
Uninstalling googletrans-3.0.0:
  Successfully uninstalled googletrans-3.0.0


En primer lugar accedemos a las urls con las listas de sancionados de EEUU en formato .csv y .xml y las guardamos en local. Cada vez que este proceso es ejecutado, se borran los archivos antiguos y se vuelven a cargar en nuestro directorio local.

In [26]:
urls = ['https://www.treasury.gov/ofac/downloads/sdn.csv',
        'https://www.treasury.gov/ofac/downloads/add.csv',
        'https://www.treasury.gov/ofac/downloads/alt.csv',
        'https://www.treasury.gov/ofac/downloads/sdn.xml']

path = 'UE_EEUU_data_crudo'

for url in urls:
    filename = path + '/' + os.path.basename(url) # get the full path of the file
    if os.path.exists(filename):
        os.remove(filename) # if exist, remove it directly
    wget.download(url, out=filename) # download it to the specific path.

A continuación cargamos los .csv con la información relevante de sancionados de EEUU en tres DataFrames relacionados por la columna uid_pk_sdn, que es la clave primaria del DataFrame df_sdn y la foránea del resto.

In [27]:
df_sdn = pd.read_csv('UE_EEUU_data_crudo/sdn.csv',usecols=list(range(3)),header=None).iloc[0:10409]
df_sdn.columns = ['uid_pk_sdn','LastName','sdnType']
df_sdn.head()

,uid_pk_sdn,LastName,sdnType
0,36,AEROCARIBBEAN AIRLINES,-0-
1,173,"ANGLO-CARIBBEAN CO., LTD.",-0-
2,306,BANCO NACIONAL DE CUBA,-0-
3,424,BOUTIQUE LA MAISON,-0-
4,475,CASA DE CUBA,-0-


In [28]:
df_add = pd.read_csv('UE_EEUU_data_crudo/add.csv',header=None,usecols=list(range(5))).iloc[0:15187]
df_add.columns = ['uid_fk_sdn','uid_pk_add','Address','City','Country']
df_add.head()

,uid_fk_sdn,uid_pk_add,Address,City,Country
0,36,25.0,-0-,Havana,Cuba
1,173,129.0,"Ibex House, The Minories",London EC3N 1DY,United Kingdom
2,306,199.0,Zweierstrasse 35,Zurich CH-8022,Switzerland
3,306,200.0,Avenida de Concha Espina 8,Madrid E-28036,Spain
4,306,201.0,"Dai-Ichi Bldg. 6th Floor, 10-2 Nihombashi, 2-c...",Tokyo 103,Japan


In [29]:
df_alt = pd.read_csv('UE_EEUU_data_crudo/alt.csv',header=None,usecols=list(range(4))).iloc[0:14099]
df_alt.columns = ['uid_fk_sdn','uid_pk_alt','AliasType','Alias']
df_alt.head()

,uid_fk_sdn,uid_pk_alt,AliasType,Alias
0,36,12.0,aka,AERO-CARIBBEAN
1,173,57.0,aka,AVIA IMPORT
2,306,220.0,aka,NATIONAL BANK OF CUBA
3,540,471.0,aka,COIBA
4,552,475.0,aka,CRYMSA


In [14]:
pip install googletrans==4.0.0rc1

Defaulting to user installation because normal site-packages is not writeable
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17429 sha256=0f332de05fa21377afa74d6607e9c9a241ec968bb467e3c8ed10ba4c5322f258
  Stored in directory: c:\users\usuario\appdata\local\pip\cache\wheels\60\b3\27\d8aff3e2d5c2d0d97a117cdf0d5f13cd121e2c2b5fb49b55a0
Successfully built googletrans
Note: you may need to restart the kernel to use updated packages.


Para las listas de la UE, la petición a la URL se hace con un token. No sirve el código utilizado anteriormente porque no acepta el método wget.download. Este proceso no almacena la tabla en local, si no que genera directamente en Python un archivo con los datos. 

In [3]:
import csv
import urllib.request
from io import StringIO

url = 'https://webgate.ec.europa.eu/fsd/fsf/public/files/csvFullSanctionsList_1_1/content?token=n008rnnv'
respuesta = urllib.request.urlopen(url)
f = StringIO(bytearray(respuesta.read()).decode())
archivo = csv.reader(f)

A continuación transformamos el archivo generado a DataFrame.

In [6]:
df = pd.DataFrame(archivo)
df.rename(columns={0:'column'},inplace=True)
df = df[['column']]
df = df.column.str.split(pat=';',expand=True)

Pasamos los datos en crudo de la lista de la UE a local desde el Dataframe sin depurar. Cada vez que se ejecute el proceso, se elimina el .xlsx con los datos en crudo antiguos y se cargan los nuevos datos en crudo.

In [10]:
filename = path + '/' + 'sancionlist_UE.csv' # get the full path of the file
if os.path.exists(filename):
    os.remove(filename) # if exist, remove it directly

df.to_csv('UE_EEUU_data_crudo/sancionlist_UE.csv')

Depuramos el DataFrame para poner nombre a las columnas y quedarnos solo con las columnas que nos interesan.

In [30]:
df_ue = df.iloc[1:,[8,19,34,43]].rename(columns={8:'SanctionType',19:'NameAlias',
                                                 34:'AddressCity',43:'AddressCountry'})

df_ue = df_ue[df_ue['NameAlias']!='']
df_ue

,SanctionType,NameAlias,AddressCity,AddressCountry
1,person,Saddam Hussein Al-Tikriti,,
2,person,Abu Ali,,
3,person,Abou Ali,,
6,person,Qusay Saddam Hussein Al-Tikriti,None,None
7,person,Qoussaï Saddam Hussein Al-Tikriti,,
...,...,...,...,...
21515,enterprise,Grupo de Empresas FORSS,,
21516,enterprise,FORSS-Konzern,,
21517,enterprise,grupo de empresas FORSS,,
21518,enterprise,Консорциум ФОРСС,,


In [21]:
df_ue.describe()

,SanctionType,NameAlias,AddressCity,AddressCountry
count,14572,14529,13690,13690
unique,12,14280,51,1
top,person,SYR,,
freq,9928,36,13535,13690


In [20]:
df_ue.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15560 entries, 1 to 21519
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   SanctionType    14572 non-null  object
 1   NameAlias       14529 non-null  object
 2   AddressCity     13690 non-null  object
 3   AddressCountry  13690 non-null  object
dtypes: object(4)
memory usage: 607.8+ KB


Finalmente exportamos todos los DataFrame depurados en formato .xlsx a la carpeta local 'UE_EEUU_data_depurada'. Cada vez que se ejecute el proceso, se eliminan los sheets con los datos depurados antiguos y se le cargan los nuevos.

In [10]:
dfs_xlsx = ['df_sdn.xlsx','df_add.xlsx','df_alt.xlsx','df_ue.xlsx']

path = 'UE_EEUU_data_depurada'


filename = path + '/' + 'sanctionlist.xlsx' # get the full path of the file

if os.path.exists(filename):
    os.remove(filename) # if exist, remove it directly

with pd.ExcelWriter('UE_EEUU_data_depurada/sanctionlist.xlsx', engine='xlsxwriter') as writer:
    
    df_sdn.to_excel(writer, sheet_name='EEUU Name & Sanction Type')
    df_add.to_excel(writer, sheet_name='EEUU Location Information')
    df_alt.to_excel(writer, sheet_name='EEUU Alias Information')
    df_ue.to_excel(writer, sheet_name='UE All Information')


In [12]:
df_ue["NameAlias"].duplicated()

1        False
2        False
3        False
4        False
5         True
         ...  
21516    False
21517    False
21518    False
21519    False
21520     True
Name: NameAlias, Length: 21520, dtype: bool

In [5]:
df_ue=pd.read_csv('UE_EEUU_data_crudo/sancionlist_UE.csv')

C:\Users\usuario\AppData\Local\Temp\ipykernel_12308\1909085301.py:1: DtypeWarning: Columns (6,57,58,59,60,63,69,87,89,90,93,95,96,106,108,109,119,120,121,122,123,124) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ue=pd.read_csv('UE_EEUU_data_crudo/sancionlist_UE.csv')


In [6]:
df_ue_0_15 = df_ue.iloc[:,0:15]
df_ue_0_15.head()

,Unnamed: 0,﻿fileGenerationDate,Entity_LogicalId,Entity_EU_ReferenceNumber,Entity_UnitedNationId,Entity_DesignationDate,Entity_DesignationDetails,Entity_Remark,Entity_SubjectType,Entity_SubjectType_ClassificationCode,Entity_Regulation_Type,Entity_Regulation_OrganisationType,Entity_Regulation_PublicationDate,Entity_Regulation_EntryIntoForceDate,Entity_Regulation_NumberTitle
0,1,02/08/2022,13,EU.27.28,NaN,NaN,NaN,(UNSC RESOLUTION 1483),P,person,regulation,commission,2003-07-08,2003-07-07,1210/2003 (OJ L169)
1,2,02/08/2022,13,EU.27.28,NaN,NaN,NaN,(UNSC RESOLUTION 1483),P,person,regulation,commission,2003-07-08,2003-07-07,1210/2003 (OJ L169)
2,3,02/08/2022,13,EU.27.28,NaN,NaN,NaN,(UNSC RESOLUTION 1483),P,person,regulation,commission,2003-07-08,2003-07-07,1210/2003 (OJ L169)
3,4,02/08/2022,13,EU.27.28,NaN,NaN,NaN,(UNSC RESOLUTION 1483),P,person,regulation,commission,2003-07-08,2003-07-07,1210/2003 (OJ L169)
4,5,02/08/2022,13,EU.27.28,NaN,NaN,NaN,(UNSC RESOLUTION 1483),P,person,regulation,commission,2003-07-08,2003-07-07,1210/2003 (OJ L169)


In [7]:
df_ue_0_15.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21786 entries, 0 to 21785
Data columns (total 15 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   Unnamed: 0                             21786 non-null  int64 
 1   ﻿fileGenerationDate                    21786 non-null  object
 2   Entity_LogicalId                       21786 non-null  int64 
 3   Entity_EU_ReferenceNumber              21786 non-null  object
 4   Entity_UnitedNationId                  623 non-null    object
 5   Entity_DesignationDate                 16068 non-null  object
 6   Entity_DesignationDetails              742 non-null    object
 7   Entity_Remark                          17906 non-null  object
 8   Entity_SubjectType                     20795 non-null  object
 9   Entity_SubjectType_ClassificationCode  20767 non-null  object
 10  Entity_Regulation_Type                 20767 non-null  object
 11  Entity_Regulati

In [8]:
df_ue['Entity_SubjectType_ClassificationCode'].describe()

count      20767
unique        15
top       person
freq       15157
Name: Entity_SubjectType_ClassificationCode, dtype: object

In [1]:
df_ue['Entity_SubjectType_ClassificationCode'].unique()

NameError: name 'df_ue' is not defined

In [9]:
df_ue['Entity_SubjectType_ClassificationCode'].isnull().tolist().count(True)

1791

In [10]:
print("La columna con la información del tipo de entidad es 'Entity_SubjectType_ClassificationCode'.\n")

print("Se trata de una variable categórica nominal con",
      len(df_ue['Entity_SubjectType_ClassificationCode'].unique()),"valores únicos.\n")

print("Existen",df_ue['Entity_SubjectType_ClassificationCode'].isnull().tolist().count(True),
      "valores nulos de un total de",len(df_ue),'valores.')


La columna con la información del tipo de entidad es 'Entity_SubjectType_ClassificationCode'.

Se trata de una variable categórica nominal con 3 valores únicos.

Existen 1791 valores nulos de un total de 21520 valores.


In [21]:
df_ue_15_30 = df_ue.iloc[:,15:30]
df_ue_15_30.head()

,Entity_Regulation_Programme,Entity_Regulation_PublicationUrl,NameAlias_LastName,NameAlias_FirstName,NameAlias_MiddleName,NameAlias_WholeName,NameAlias_NameLanguage,NameAlias_Gender,NameAlias_Title,NameAlias_Function,NameAlias_LogicalId,NameAlias_RegulationLanguage,NameAlias_Remark,NameAlias_Regulation_Type,NameAlias_Regulation_OrganisationType
0,IRQ,http://eur-lex.europa.eu/LexUriServ/LexUriServ...,Hussein Al-Tikriti,Saddam,NaN,Saddam Hussein Al-Tikriti,NaN,M,NaN,NaN,17.0,EN,NaN,regulation,commission
1,IRQ,http://eur-lex.europa.eu/LexUriServ/LexUriServ...,NaN,NaN,NaN,Abu Ali,NaN,NaN,NaN,NaN,19.0,EN,NaN,regulation,commission
2,IRQ,http://eur-lex.europa.eu/LexUriServ/LexUriServ...,NaN,NaN,NaN,Abou Ali,FR,NaN,NaN,NaN,380.0,EN,NaN,regulation,commission
3,IRQ,http://eur-lex.europa.eu/LexUriServ/LexUriServ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,IRQ,http://eur-lex.europa.eu/LexUriServ/LexUriServ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df_ue_15_30.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21520 entries, 0 to 21519
Data columns (total 15 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Entity_Regulation_Programme            19729 non-null  object 
 1   Entity_Regulation_PublicationUrl       19729 non-null  object 
 2   NameAlias_LastName                     1796 non-null   object 
 3   NameAlias_FirstName                    1790 non-null   object 
 4   NameAlias_MiddleName                   863 non-null    object 
 5   NameAlias_WholeName                    14216 non-null  object 
 6   NameAlias_NameLanguage                 5765 non-null   object 
 7   NameAlias_Gender                       4507 non-null   object 
 8   NameAlias_Title                        430 non-null    object 
 9   NameAlias_Function                     2553 non-null   object 
 10  NameAlias_LogicalId                    13505 non-null  float64
 11  Na

In [25]:
len(df_ue_15_30['NameAlias_WholeName'].unique())

14040

In [26]:
print("La columna con la información del nombre de la entidad es 'NameAlias_WholeName'.\n")

print("Se trata de una variable categórica nominal con",
      len(df_ue['NameAlias_WholeName'].unique()),"valores únicos.\n")

print("Existen",df_ue['NameAlias_WholeName'].isnull().tolist().count(True),
      "valores nulos de un total de",len(df_ue),'valores.')

La columna con la información del nombre de la entidad es 'NameAlias_WholeName'.

Se trata de una variable categórica nominal con 14040 valores únicos.

Existen 7304 valores nulos de un total de 21520 valores.


In [4]:
df_ue_30_45 = df_ue.iloc[:,30:45]
df_ue_30_45.head()

,NameAlias_Regulation_PublicationDate,NameAlias_Regulation_EntryIntoForceDate,NameAlias_Regulation_NumberTitle,NameAlias_Regulation_Programme,NameAlias_Regulation_PublicationUrl,Address_City,Address_Street,Address_PoBox,Address_ZipCode,Address_Region,Address_Place,Address_AsAtListingTime,Address_ContactInfo,Address_CountryIso2Code,Address_CountryDescription
0,2003-07-08,2003-07-07,1210/2003 (OJ L169),IRQ,http://eur-lex.europa.eu/LexUriServ/LexUriServ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2003-07-08,2003-07-07,1210/2003 (OJ L169),IRQ,http://eur-lex.europa.eu/LexUriServ/LexUriServ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2003-07-08,2003-07-07,1210/2003 (OJ L169),IRQ,http://eur-lex.europa.eu/LexUriServ/LexUriServ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df_ue_30_45.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21520 entries, 0 to 21519
Data columns (total 15 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   NameAlias_Regulation_PublicationDate     13450 non-null  object
 1   NameAlias_Regulation_EntryIntoForceDate  13450 non-null  object
 2   NameAlias_Regulation_NumberTitle         13450 non-null  object
 3   NameAlias_Regulation_Programme           13445 non-null  object
 4   NameAlias_Regulation_PublicationUrl      13445 non-null  object
 5   Address_City                             826 non-null    object
 6   Address_Street                           510 non-null    object
 7   Address_PoBox                            32 non-null     object
 8   Address_ZipCode                          57 non-null     object
 9   Address_Region                           33 non-null     object
 10  Address_Place                            24 non-null     o

In [30]:
len(df_ue_30_45['Address_City'].unique())

335

In [35]:
len(df_ue_30_45['Address_CountryDescription'].unique())

60

In [37]:
print("Las columnas con la información de la ubicación son 'Address_City' y 'Address_CountryDescription'.\n")

print("Se tratan de variables categóricas nominales con",
      len(df_ue['Address_City'].unique()),'y',len(df_ue_30_45['Address_CountryDescription'].unique()),
      "valores únicos, respectivamente.\n")

print("Existen",df_ue['Address_City'].isnull().tolist().count(True),
      "valores nulos para 'Address_City' y",df_ue['Address_CountryDescription'].isnull().tolist().count(True),
      "para 'Address_CountryDescription' de un total de",len(df_ue),"valores.")

Las columnas con la información de la ubicación son 'Address_City' y 'Address_CountryDescription'.

Se tratan de variables categóricas nominales con 335 y 60 valores únicos, respectivamente.

Existen 20694 valores nulos para 'Address_City' y 20947 para 'Address_CountryDescription' de un total de 21520 valores.


Una vez identificadas las variables de interés de la lista de la UE, se crea el DataFrame df_ue_t1 con estas variables. El 't1' en el nombre implica que esta es la primera transformación hecha sobre los datos tras ser cargados. Este DataFrame:

- Presenta una nueva variable llamada 'Origin' de valor 'UE' para identificar cual es la lista de origen.
- Se le han eliminado los registros con 'NameAlias_WholeName' nulo. Ya que no podemos identificar un sancionado si no tiene un nombre.
- En caso de existir, se eliminan los registros duplicados. Para que también se eliminen los registros duplicados pero que se diferencian por las mayúculas y minúsculas, se ponen todas la variables de tipo texto en mayúsculas.
- Se cambia el nombre de las variables para darles un nombre más conciso.

In [6]:
df_ue_t1 = df_ue.loc[:,['Entity_SubjectType_ClassificationCode','NameAlias_WholeName',
                        'Address_City','Address_CountryDescription']]

df_ue_t1.rename(columns={'Entity_SubjectType_ClassificationCode':'EntityType',
                'NameAlias_WholeName':'Name','Address_City':'City',
                'Address_CountryDescription':'Country'},inplace=True)

df_ue_t1.dropna(subset=['Name'],inplace=True)

df_ue_t1['Origin'] = 'UE'

df_ue_t1['EntityType'] = df_ue_t1['EntityType'].str.upper()
df_ue_t1['Name'] = df_ue_t1['Name'].str.upper()
df_ue_t1['City'] = df_ue_t1['City'].str.upper()
df_ue_t1['Country'] = df_ue_t1['Country'].str.upper()
df_ue_t1.drop_duplicates(inplace=True)
df_ue_t1.reset_index(inplace=True)
df_ue_t1.drop(columns=['index'],axis=1,inplace=True)

df_ue_t1


,EntityType,Name,City,Country,Origin
0,PERSON,SADDAM HUSSEIN AL-TIKRITI,NaN,NaN,UE
1,PERSON,ABU ALI,NaN,NaN,UE
2,PERSON,ABOU ALI,NaN,NaN,UE
3,PERSON,QUSAY SADDAM HUSSEIN AL-TIKRITI,NaN,NaN,UE
4,PERSON,QOUSSAÏ SADDAM HUSSEIN AL-TIKRITI,NaN,NaN,UE
...,...,...,...,...,...
14009,ENTERPRISE,GRUPUL DE SOCIETĂȚI FORSS,NaN,NaN,UE
14010,ENTERPRISE,GRUPO DE EMPRESAS FORSS,NaN,NaN,UE
14011,ENTERPRISE,FORSS-KONZERN,NaN,NaN,UE
14012,ENTERPRISE,КОНСОРЦИУМ ФОРСС,NaN,NaN,UE


Aunque los duplicados han sido eliminados, existen registros muy parecidos que solo se distinguen entre ellos por pequeñas diferencias en el nombre. Estos registros pueden considerarse duplicados. Para eliminarlos, se emplea la función 'get_close_matches()' del módulo de Python llamado difflib.

La función se emplea dentro de un bucle for que comprueba para cada nombre cuantos nombres similares tiene debajo. Si se detecta que no hay ningún nombre similar, este se registro es mandado a la lista 'new_list_name' con un condicional de tipo if. Esta función presenta dos parámetros ajustables:

- 'n' tiene valor 4 por defecto e indica el número máximo de registros que, definido un cierto nombre, pueden ser similares a este. Es decir, para cada nombre el número máximo de nombres que pueden considerarse duplicados debido a su similitud y por tanto pueden ser eliminados es de 4 nombres.


- 'sensibility' es un número entre 0 y 1 que indica el grado de sensibilidad con el que funciona el detector de matches. A mayor sensibilidad, más registros serán considerados como duplicados y, en consecuencia, menos nombres aparecen en la lista final.

In [ ]:
list_name = df_ue_t1['Name']
new_list_name = []

n = 4
sensibility = 0.2

for i in list(range(len(list_name))):
    
    word = list_name[i]
    possibilities = list_name[(i+1):]
    
    close_matches = difflib.get_close_matches(word, possibilities, n, 1-sensibility)
    
    if len(close_matches) == 0:
        new_list_name.append(list_name[i])
new_list_name

In [74]:
len(new_list_name)

8533

In [76]:
new_df_name = pd.DataFrame(new_list_name, columns=['Name'])

pd.merge(new_df_name,df_ue_t1,how='inner',on='Name')

,Name,EntityType,City,Country,Origin
0,ABDEL HAMID MAHMOUD,PERSON,NaN,NaN,UE
1,ABID HAMID MAHMOUD AL-TIKRITI,PERSON,NaN,NaN,UE
2,ABID HAMID BID HAMID MAHMOUD,PERSON,NaN,NaN,UE
3,ABED MAHMOUD HAMMOUD,PERSON,NaN,NaN,UE
4,ALI HASSAN AL-MAJID AL-TIKRITI,PERSON,NaN,NaN,UE
...,...,...,...,...,...
8529,SKUPINA SPOLOČNOSTÍ FORSS,ENTERPRISE,NaN,NaN,UE
8530,GRUPUL DE SOCIETĂȚI FORSS,ENTERPRISE,NaN,NaN,UE
8531,GRUPO DE EMPRESAS FORSS,ENTERPRISE,NaN,NaN,UE
8532,КОНСОРЦИУМ ФОРСС,ENTERPRISE,NaN,NaN,UE


In [5]:
text = 'GRUPUL DE SOCIETĂȚI FORSS'
from googletrans import Translator
translator = Translator() 
translator.translate(text , dest ='es').text 

'Grupo de empresas Forss'